### Using Dogleg() for optimization

In [ ]:
import CSV
using LeastSquaresOptim, DifferentialEquations, DelayDiffEq, DiffEqBase, Optim
data = CSV.read("taxol1.csv")
total = CSV.read("taxol1_pop.csv")

total_old = total[:,3]
G2_old = data[:,3]

estim_init = [1.0, 0.7] # [init_g1, init_g2]
# rescaling the experimental data
total_new = (estim_init[1] + estim_init[2]) * total_old
G2_new = 0.01*total_new.*G2_old
G1_new = total_new - G2_new

# This model doesn't assume delay for dying
function G1_G2(du, u, h, p, t)
    du[1] = -p[1]*(h(p, t-p[5])[1]) + 2*p[2]*(h(p, t-p[6])[2]) - p[3]*u[1]
    du[2] = p[1]*(h(p, t-p[5])[1]) - p[2]*(h(p, t-p[6])[2]) - p[4]*u[2] 
end

function solution(pp)
    lags = [pp[5], pp[6]]
    h(p, t) = p[9]*ones(2)
    t = LinRange(0.0, 95.5, 192)
    
    tspan = (0.0, 95.5)
    u0 = [pp[7], pp[8]]
    prob = DDEProblem(G1_G2, [pp[7], pp[8]], h, tspan, pp; constant_lags = lags)
    solve(prob, MethodOfSteps(Tsit5()), lower = [0.0, 0.0])
end

function resid(pp)
    t = LinRange(0.0, 95.5, 192)
    res = zeros(2, length(t))
    sol = solution(pp)
    res[1,:] = sol(t, idxs=1) - G1_new
    res[2,:] = sol(t, idxs=2) - G2_new
    return res
end

p =[0.6007846901450706,0.3048539582338586,1.4065355422287749,0.7570231965413762,10.527938210733288,6.598877610124149,0.996591276073392,0.720611090806896,10.7637016702848363]
lowwer = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
# upp = [3.0, 3.0, 3.0, 3.0, 30.0, 30.0, 10.0, 10.0, 10.0]
results = optimize(resid, p, Dogleg(), lower = lowwer, iterations = 10000)

### Solve and Plot for Dogleg() optimization

In [ ]:
params = results.minimizer
total_n = (params[7] + params[8]) * total_old
G2_n = 0.01*total_n.*G2_old
G1_n = total_n - G2_n

sol = solution(params)
t = LinRange(0.0, 95.5, 192)

using Plots; 
plot(t, sol(t, idxs=2).u, label = "G2 est", title = "taxol1 Trial 3 Dogleg()")
plot!(t, G2_n, label = "G2 new")
plot!(t, (sol(t, idxs=2).u + sol(t, idxs=1).u), label = "total est")
plot!(t, total_n, label = "total new")
plot!(t, sol(t, idxs=1).u, label = "G1 est")
savefig("taxol3_Dogleg_.png")

### Optimization using Simulated Annealing

In [ ]:
import CSV
using LeastSquaresOptim, DifferentialEquations, DelayDiffEq
using DiffEqBase, Optim, LinearAlgebra, Plots
data = CSV.read("DOX.csv")
total = CSV.read("DOX_pop.csv")

total_old = total[48:end,5]
G2_old = data[48:end,5]

estim_init = [2.0, 1.0] # [init_g1, init_g2]
# rescaling the experimental data
total_new = (estim_init[1] + estim_init[2]) * total_old
G2_new = 0.01*total_new.*G2_old
G1_new = total_new - G2_new
 
# This model doesn't assume delay for dying
function G1_G2(du, u, h, p, t)
    du[1] = -p[1]*(h(p, t-p[5])[1]) + 2*p[2]*(h(p, t-p[6])[2]) - p[3]*u[1]
    du[2] = p[1]*(h(p, t-p[5])[1]) - p[2]*(h(p, t-p[6])[2]) - p[4]*u[2] 
end

function solution(pp)
    lags = [pp[5], pp[6]]
    h(p, t) = pp[9]*ones(2)
    t = LinRange(0.0, 71.5, 145)
    tspan = (0.0, 71.5)
    u0 = [pp[7], pp[8]]

    prob = DDEProblem(G1_G2, [pp[7], pp[8]], h, tspan, pp; constant_lags = lags)
    sol = solve(prob, MethodOfSteps(Tsit5()), lower = [0.0, 0.0], upper = [20.0, 20.0])
end

function resid(pp)
    t = LinRange(0.0, 71.5, 145)
    sol = solution(pp)
    res = sol(t, idxs=2) - G2_new
    
    return norm(res)
end

p = [0.31,0.38,0.28,0.10,4.55,4.76,0.66,0.57,0.65]
lowwer = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
upperr = [1.0, 2.0, 2.0, 2.0, 15.0, 15.0, 2.0, 2.0, 2.0]

res = Optim.optimize(resid, lowwer, upperr, p, SAMIN(rt = 0.8), Optim.Options(iterations = 10^6), show_trace = true)


### Solve after SAMIN() optimization

In [ ]:
par = res.minimizer
total_nw = (par[7] + par[8]) * total_old
G2_new = 0.01*total_nw.*G2_old
G1_new = total_nw - G2_new
final = solution(par)
t = LinRange(0.0, 71.5, 145)

using Plots;
plot(t, final(t, idxs=2).u, label = "G2 est", title = "Dox Trial 3 samin(rt=0.7)")
plot!(t, G2_new, label = "G2")
plot!(t, (final(t, idxs=2).u + final(t, idxs=1).u), label = "total est")
plot!(t, total_nw, label = "total")
plot!(t, final(t, idxs=1).u, label = "est G1")
savefig("DOX_one_5_samin_rt7_24cut.png")